In [1]:
import os
import subprocess

import numpy as np

from hyperopt import hp
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

import Augmentor
import json

import tuner
from tuner import utils
from tuner import load_data
from tuner import augment_data
from tuner import tune_cnn
from tuner import use_hyperas
from tuner import net

from bson.objectid import ObjectId

Using TensorFlow backend.
/home/fytroo/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class StandardDataset(object):
    def __init__(self, dataset_dir):
        self._id = ObjectId()
        self.id = str(self._id)
        self.size = 96
        self.scale = 1.
        self.original_dataset_path = dataset_dir
        self.path = 'standard_datasets/{}'.format(self.id)
        self.train_dir = os.path.join(self.path, 'train') 
        self.validation_dir = os.path.join(self.path, 'validation') 
        
        utils.mkdir(self.path)
        load_data.format_dataset(
            self.original_dataset_path, self.path, mode='eyes')
        
        self.df_train = load_data.df_fromdir(self.train_dir)
        self.df_validation = load_data.df_fromdir(self.validation_dir)
        
    def counts_train_data(self):
        return self.df_train['label'].value_counts().to_dict()
    
    def counts_validation_data(self):
        return self.df_validation['label'].value_counts().to_dict()
    
    def load_train_data(self):
        df = load_data.df_fromdir(self.train_dir)
        x_train, y_train = load_data.load_fromdf(df, resize=self.size, rescale=self.scale)
        return x_train, y_train
        
    def load_validation_data(self):
        df = load_data.df_fromdir(self.validation_dir)
        x_val, y_val = load_data.load_fromdf(df, resize=self.size, rescale=self.scale)
        return x_val, y_val

In [3]:
brain_dataset = StandardDataset('./brain')

In [4]:
min(brain_dataset.counts_train_data().values())

46

In [5]:
class AugmentDataset(object):
    def __init__(self, standard_dataset):
        self._id = ObjectId()
        self.id = str(self._id)
        self.dataset = standard_dataset
        self.augment_condition = 'cond.json'
        
    def search_opt_augment(self, model=net.aug):
        best_condition, best_model = use_hyperas.exec_hyperas(
            self.dataset.train_dir,
            self.dataset.validation_dir,
            model)
        with open(self.augment_condition, 'w') as f:
            json.dump(best_condition, f)
            
    def augment_dataset(self, sampling_size=None):
        self.augmented_dir = os.path.join(self.dataset.path, 'auged')
        sampling_size =\
            sampling_size if sampling_size else\
            min(self.dataset.counts_train_data().values()) * 4
        augment_data.augment_dataset(
            self.dataset.train_dir,
            self.augmented_dir,
            condition_file=self.augment_condition,
            sampling_size=sampling_size
        )
        
    def search_opt_cnn(self, model=net.simplenet):
        best_condition, best_model = use_hyperas.exec_hyperas(
            self.dataset.train_dir,
            self.dataset.validation_dir,
            model)
        fname = 'simplenet.hdf5'
        best_model.save(fname)
        return fname

In [6]:
aug_brain = AugmentDataset(brain_dataset)

In [9]:
aug_brain.search_opt_augment(model=net.aug)

>>> Imports:
#coding=utf-8

try:
    import subprocess
except:
    pass

try:
    from tuner import net
except:
    pass

try:
    from hyperopt import hp
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers import BatchNormalization, Flatten, Dense
except:
    pass

try:
    from keras.layers import Input, Conv2D, Convolution2D
except:
    pass

try:
    from keras.layers import Activation, concatenate, Dropout
except:
    pass

try:
    from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
except:
    pass

try:
    from keras.models import Sequential

Epoch 1/10


KeyboardInterrupt: 

In [8]:
aug_brain.augment_dataset()

Processing MS125.jpg:   5%|▍         | 9/184 [00:00<00:02, 58.54 Samples/s]

standard_datasets/5a4e3a66b037cb9b8e2ef254/train standard_datasets/5a4e3a66b037cb9b8e2ef254/auged
Initialised with 46 image(s) found.
Output directory set to standard_datasets/5a4e3a66b037cb9b8e2ef254/train/MS/output.

Processing PD057.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.89 Samples/s]  

Initialised with 46 image(s) found.
Output directory set to standard_datasets/5a4e3a66b037cb9b8e2ef254/train/PD/output.

Processing N080.jpg:  10%|█         | 19/184 [00:00<00:01, 102.34 Samples/s] 

Initialised with 79 image(s) found.
Output directory set to standard_datasets/5a4e3a66b037cb9b8e2ef254/train/N/output.

Processing PS107.jpg:   3%|▎         | 5/184 [00:00<00:05, 32.97 Samples/s] 

Initialised with 75 image(s) found.
Output directory set to standard_datasets/5a4e3a66b037cb9b8e2ef254/train/PS/output.

In [15]:
def search_opt_cnn(train_dir, validation_dir, model):
    best_condition, best_model = use_hyperas.exec_hyperas(
        train_dir,
        validation_dir,
        model)
    fname = 'simplenet.hdf5'
    best_model.save(fname)
    return fname

In [17]:
search_opt_cnn(
    aug_brain.augmented_dir,
    aug_brain.dataset.validation_dir,
    net.simplenet
)

>>> Imports:
#coding=utf-8

try:
    import subprocess
except:
    pass

try:
    from tuner import net
except:
    pass

try:
    from hyperopt import hp
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    import tensorflow as tf
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers import BatchNormalization, Flatten, Dense
except:
    pass

try:
    from keras.layers import Input, Conv2D, Convolution2D
except:
    pass

try:
    from keras.layers import Activation, concatenate, Dropout
except:
    pass

try:
    from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
except:
    pass

try:
    from keras.models import Sequential

Epoch 54/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 55/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 56/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 57/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 58/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 59/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 60/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 61/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 62/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 63/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 64/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 65/100
 - 0s - loss: 12.0886 - acc: 0

Epoch 48/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 49/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 50/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 51/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 52/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 53/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 54/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 55/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 56/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 57/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 58/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 59/100
 - 0s - loss: 12.0886 - acc: 0

Epoch 42/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 43/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 44/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 45/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 46/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 47/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 48/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 49/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 50/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 51/100
 - 1s - loss: 12.1105 - acc: 0.2486 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 52/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 53/100
 - 1s - loss: 12.0886 - acc: 0

Epoch 36/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 37/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 38/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 39/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 40/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 41/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 42/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 43/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 44/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 45/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 46/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 47/100
 - 1s - loss: 12.0886 - acc: 0

Epoch 30/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 31/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 32/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 33/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 34/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 35/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 36/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 37/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 38/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 39/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 40/100
 - 1s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 41/100
 - 1s - loss: 12.0886 - acc: 0

Epoch 24/100
 - 0s - loss: 12.1324 - acc: 0.2473 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 25/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 26/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 27/100
 - 0s - loss: 12.1324 - acc: 0.2473 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 28/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 29/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 30/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 31/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 32/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 33/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 34/100
 - 0s - loss: 12.0886 - acc: 0.2500 - val_loss: 12.0886 - val_acc: 0.2500
Epoch 35/100
 - 0s - loss: 12.0886 - acc: 0

KeyboardInterrupt: 